Pu pratices his Pytorch skills by writing a classification algorithm using Pytorch. He also uses sklearn's LabelEncoder to transform categorical features. This is based on the tutorial on this page: https://machinelearningmastery.com/use-pytorch-deep-learning-models-with-scikit-learn/

In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch

# read the data
data = pd.read_csv("../data/label-encoder/sonar.csv")
X = data.iloc[:, :60]
y = data.iloc[:, 60]

encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)


X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [35]:
# define the model
import torch.nn as nn

class SonarClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(60, 60)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(60, 60)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(60, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.output(x)
        return x

In [36]:
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

batch_size = 5
learning_rate = 0.01
num_epochs = 50

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, shuffle=True)
train, test = TensorDataset(X_train, y_train), TensorDataset(X_test, y_test)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

model = SonarClassifier()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.BCEWithLogitsLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device).view(-1, 1)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(f'Epoch: [{epoch} / {num_epochs}], loss: {loss.item()}')
    model.eval()
    with torch.no_grad():
        all_preds, all_labels = [], []
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device).view(-1, 1)
            outputs = model(inputs)
            predicted = torch.round(torch.sigmoid(outputs))
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
            num_correct += (predicted == targets).float().sum().item()
            num_samples += targets.size(0)

        precision = precision_score(all_labels, all_preds)
        recall = recall_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds)
        accuracy = accuracy_score(all_labels, all_preds)
        print(f"Accuracy: {accuracy * 100:.2f}%, Precision: {precision * 100:.2f}%, Recall: {recall * 100:.2f}%, F1: {f1 * 100:.2f}%")


Epoch: [0 / 50], loss: 0.6961638331413269
Epoch: [0 / 50], loss: 0.7821555733680725
Accuracy: 41.10%, Precision: 41.10%, Recall: 100.00%, F1: 58.25%
Epoch: [1 / 50], loss: 0.6077275276184082
Epoch: [1 / 50], loss: 0.6303961873054504
Accuracy: 41.10%, Precision: 41.10%, Recall: 100.00%, F1: 58.25%
Epoch: [2 / 50], loss: 0.7696098685264587
Epoch: [2 / 50], loss: 0.5698317885398865
Accuracy: 41.10%, Precision: 41.10%, Recall: 100.00%, F1: 58.25%
Epoch: [3 / 50], loss: 0.6237584948539734
Epoch: [3 / 50], loss: 0.5406121611595154
Accuracy: 41.78%, Precision: 41.38%, Recall: 100.00%, F1: 58.54%
Epoch: [4 / 50], loss: 0.5494301319122314
Epoch: [4 / 50], loss: 0.8884040713310242
Accuracy: 71.92%, Precision: 62.03%, Recall: 81.67%, F1: 70.50%
Epoch: [5 / 50], loss: 0.2347792685031891
Epoch: [5 / 50], loss: 0.2968374788761139
Accuracy: 63.70%, Precision: 53.33%, Recall: 93.33%, F1: 67.88%
Epoch: [6 / 50], loss: 0.5366269946098328
Epoch: [6 / 50], loss: 0.2955125868320465
Accuracy: 70.55%, Precis